In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
#import seaborn as sns
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.decomposition import PCA
from feature_engineering import *
from train_and_pred import *
from score import *


# Read File

In [2]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv') #Data Test for kaggle

### Replace values where NaN has meaning

In [3]:
data = replace_NaN_meaning(data)
data_test = replace_NaN_meaning(data_test)

### Removing columns that has the same value in all rows

In [4]:
data = remove_useless_features(data)
data_test = remove_useless_features(data_test)

### Ading new features

In [5]:
data = adding_features(data)
data_test = adding_features(data_test)

## Transforming some numerical variables that are really categorical

In [6]:
data = transform_numerical_col_categorical(data)
data_test = transform_numerical_col_categorical(data_test)

### Split features and target

In [7]:
lis_drop_num_columns = ['SalePrice','Id']
X = data.drop(lis_drop_num_columns,axis=1)
Y = data[['SalePrice']]

id_test_kaggle = data_test[['Id']]
X_kaggle = data_test.drop('Id',axis=1)

# Handle Missing Data

In [8]:
X, num_columns, categ_columns = handle_missing_data(X)
X_kaggle, num_columns_kg, categ_columns_kg = handle_missing_data(X_kaggle)

## Concat Dataframes

In [9]:
concat_df = pd.concat([X,X_kaggle])

## Normalize numerical data

In [10]:
concat_df[num_columns.columns] = preprocessing.scale(concat_df[num_columns.columns])

## Using One-Hot encoding

In [11]:
#use one-hot encoding in categorical data
one_hot_encoding_all = pd.get_dummies(concat_df[categ_columns.columns])

### Join categorical and numerical columns again

In [12]:
X_final, X_final_kaggle = split_data_dataKaggle(X, X_kaggle, one_hot_encoding_all,num_columns)

In [15]:
X_final_kaggle.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,1,0,0,0,0,1,0
1,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,1,0,0,0,0,1,0
2,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,1,0,0,0,0,1,0
3,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,...,0,0,0,1,0,0,0,0,1,0
4,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,...,0,0,0,1,0,0,0,0,1,0


## Feature Selection

In [17]:
print(X_final.shape)
#X_final, X_final_kaggle = feature_selection(X_final, X_final_kaggle, 'VarianceThreshold', porcentage=0.99)
#X_final, X_final_kaggle, var = feature_selection(X_final, X_final_kaggle, 'PCA', n_comp=300)
#X_final, X_final_kaggle = feature_selection(X, X_kaggle, 'SelectKBest', k =200, y = Y)
print(X_final.shape)


(1460, 322)
(1460, 322)


## Split train and test

In [ ]:
#Split train x test
X_train, X_test, y_train, y_test = train_test_split(X_final, Y, shuffle=True, random_state=42)

# Train and prediction

#### Random Florest

In [ ]:
reg1, y_pred1 = train_pred_RandomFlorest(X_train, y_train, X_test, grid_search=True)

#### Gradient Boosting Regressor

In [ ]:
reg2, y_pred2 = train_pred_GradientBoostingRegressor(X_train, y_train, X_test, grid_search=True)

### XGboost

In [ ]:
reg3, y_pred3 = train_pred_XGboost(X_train, y_train, X_test, grid_search=True)

## Final Predictor to Combine the predictors

In [ ]:
# Train
list_alg = ['rf', 'gb', 'xgboost']
list_predictors = [reg1, reg2, reg3]
reg_comb = train_comb_predictor(X_train, y_train, list_alg, list_predictors)

#Test
list_predictions = [y_pred1, y_pred2, y_pred3, y_pred3]
y_pred_ens = test_comb_predictor(reg_comb, list_alg, list_predictions)

In [ ]:
print(score(y_test,y_pred1))
print(score(y_test,y_pred2))
print(score(y_test,y_pred3))

## Score

In [ ]:
score(y_test,y_pred_ens)

# Test Kaggle

In [ ]:
#make sure there is no NaN values
#X_final_kaggle[num_columns.columns] = X_final_kaggle[num_columns.columns].groupby(num_columns.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))

In [ ]:
Y_kaggle_predictions = []
for reg in list_predictors:
    Y_kaggle_predictions.append(reg.predict(X_final_kaggle))

### Ensemble Predictor 

In [ ]:
Y_kaggle_ens = test_comb_predictor(reg_comb, list_alg, Y_kaggle_predictions)

In [ ]:
df_submission_kaggle = pd.DataFrame(Y_kaggle_ens, columns =['SalePrice']) 
df_submission_kaggle['Id'] = id_test_kaggle

df_submission_kaggle.to_csv('submission.csv', index=False)

df_submission_kaggle.shape